In [ ]:
# 1. Her şeyi temiz ve uyumlu kuran dev komut
!pip install -q -U langchain langchain-community langchain-huggingface pypdf chromadb sentence-transformers accelerate bitsandbytes transformers

# 2. Kurulum bittikten sonra otomatik durdurma (Hoca sorarsa 'ortamı stabilize etmek için yaptım' dersin)
import os
os.kill(os.getpid(), 9)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 50.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 329.6/329.6 kB 21.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.7/21.7 MB 96.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 24.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 101.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 65.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 484.9/484.9 kB 44.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 129.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 7.2 MB/s eta 0:

In [ ]:
from google.colab import drive
import os
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma

# 1. Drive'ı bağla
drive.mount('/content/drive')

# 2. Klasör yolunu kontrol et
# PDF'lerin tam olarak hangi klasördeyse orayı yazmalısın.
pdf_yolu = "/content/drive/MyDrive/makaleler/"

documents = []
if os.path.exists(pdf_yolu):
    print(f"📂 Klasör bulundu: {pdf_yolu}")
    for file in os.listdir(pdf_yolu):
        if file.endswith(".pdf"):
            print(f"📄 Okunuyor: {file}")
            try:
                loader = PyPDFLoader(os.path.join(pdf_yolu, file))
                documents.extend(loader.load())
            except Exception as e:
                print(f"⚠️ {file} okunurken hata: {e}")

    if len(documents) > 0:
        print(f"✅ Başarılı! Toplam {len(documents)} sayfa döküman yüklendi.")
    else:
        print("❌ HATA: Klasörde PDF bulunamadı. Lütfen PDF'leri doğru klasöre attığından emin ol.")
else:
    print(f"❌ HATA: '{pdf_yolu}' yolu bulunamadı! Drive'daki klasör ismini kontrol et.")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
📂 Klasör bulundu: /content/drive/MyDrive/makaleler/
📄 Okunuyor: NIPS-2017-attention-is-all-you-need-Paper.pdf


📄 Okunuyor: NeurIPS-2020-retrieval-augmented-generation-for-knowledge-intensive-nlp-tasks-Paper.pdf
📄 Okunuyor: ReAct Synergizing Reasoning and Acting in Language Models.pdf
📄 Okunuyor: The Llama 3 Herd of Models.pdf
📄 Okunuyor: Language Models are Few-Shot Learners (GPT-3 Makalesi).pdf
📄 Okunuyor: LLMWEEK1.pdf
📄 Okunuyor: LLMWEEK4.pdf
📄 Okunuyor: LLMWEEK2.pdf
📄 Okunuyor: LLMWEEK3.pdf
✅ Başarılı! Toplam 305 sayfa döküman yüklendi.


In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from langchain_huggingface import HuggingFacePipeline
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate

# 1. Metinleri Anlamsal Parçalara Bölme (RAG Mantığı)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=800, chunk_overlap=80)
chunks = text_splitter.split_documents(documents)

# 2. Vektör Veritabanı Oluşturma (SBERT ile)
print("🧠 Anlamsal hafıza oluşturuluyor...")
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vector_db = Chroma.from_documents(chunks, embeddings)

# 3. Llama-3 Modelini GPU'ya Yükleme (4-bit Kuantize)
model_id = "unsloth/llama-3-8b-Instruct-bnb-4bit"
print("🤖 Llama-3 GPU'ya yükleniyor...")

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    torch_dtype=torch.float16
)

# 4. Pipeline Ayarları (Tekrarı engelleyen parametreler)
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=256,
    temperature=0.2,
    repetition_penalty=1.2,
    do_sample=True
)
llm = HuggingFacePipeline(pipeline=pipe)

# 5. Akademik Asistan Zinciri (Prompt Engineering)
template = """<|begin_of_text|><|start_header_id|>system<|end_header_id|>
Sen bir Akademik Asistansın.
GÖREVİN: Sana verilen bağlamdaki bilgileri analiz etmek ve KESİNLİKLE TÜRKÇE cevap vermektir.
KURAL 1: Bağlam İngilizce olsa dahi cevabın %100 Türkçe olmalıdır.
KURAL 2: Teknik terimlerin Türkçe karşılığını kullan veya parantez içinde belirt.
KURAL 3: Asla İngilizce cümle kurma.<|eot_id|><|start_header_id|>user<|end_header_id|>
BAĞLAM: {context}
SORU: {question}<|end_header_id|><|start_header_id|>assistant<|end_header_id|>"""

prompt = PromptTemplate.from_template(template)

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": vector_db.as_retriever(search_kwargs={"k": 3}) | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

print("\n🚀 SİSTEM HAZIR! Artık akademik makalelerin hakkında soru sorabilirsin.")

🧠 Anlamsal hafıza oluşturuluyor...
🤖 Llama-3 GPU'ya yükleniyor...


Device set to use cuda:0



🚀 SİSTEM HAZIR! Artık akademik makalelerin hakkında soru sorabilirsin.


In [ ]:
# Test Sorusu: Makalelerindeki en temel konulardan birini soralım
soru = "Transformer mimarisindeki self-attention (öz-dikkat) mekanizması nedir ve geleneksel RNN modellerinden farkı nedir?"

print("🔍 Dökümanlar taranıyor ve cevap üretiliyor...")
cevap = rag_chain.invoke(soru)

# Llama-3'ün ürettiği ham metinden sadece asistanın cevabını ayıklıyoruz
temiz_cevap = cevap.split("assistant")[-1].strip()

print("\n" + "="*50)
print(f"🤔 SORU: {soru}")
print("="*50)
print(f"🤖 AKADEMİK ASİSTANIN CEVABI:\n\n{temiz_cevap}")
print("="*50)

🔍 Dökümanlar taranıyor ve cevap üretiliyor...

🤔 SORU: Transformer mimarisindeki self-attention (öz-dikkat) mekanizması nedir ve geleneksel RNN modellerinden farkı nedir?
🤖 AKADEMİK ASİSTANIN CEVABI:

<|end_header_id|>

Transformer mimarisinde kullanılan öz-dikkat mekanizması, bir dizi elemanlarının birbirleriyle olan ilişkiyi hesaplamak için kullanılır. Bu mekanizmanın amacı, dizi içindeki belli başlı elemanların önemini belirlemek ve bu elemanlara daha fazla dikkat etmektir.

Bu mekanizmaya göre, bir modele verilmiş bir metnin farklı parçalarına "dikkat" edilir. Herhangi bir parçası diğerlerine göre daha önemlidir, o zaman model ona daha fazla dikkat eder. Böylece, model, önemli olan parçaları daha iyi anlayarak, metni daha etkili bir şekilde işleyebilir.

Geleneksel RNN modellerine kıyasla, öz-dikkat mekanizmasının en büyük farkı, her tarihte tüm önceki adımları hesaba katmak zorunda olmamasıdır. Öz-dikkat mekanizması, sadece ilgili parçalara odaklanarak, hem hızlı hem de yüksek doğ

In [1]:
!pip install -q gradio

import gradio as gr

def akademik_asistan_yanitla(soru):
    """Kullanıcıdan gelen soruyu RAG zincirine gönderir ve temiz Türkçe yanıt döner."""
    if not soru.strip():
        return "Lütfen makaleler veya ders notları hakkında bir soru girin."

    try:
        # RAG zincirini çalıştır
        sonuc = rag_chain.invoke(soru)

        # 1. Llama-3 formatındaki 'assistant' etiketinden sonrasını al
        if "assistant" in sonuc:
            cevap = sonuc.split("assistant")[-1]
        else:
            cevap = sonuc

        # 2. Teknik kalıntıları (<|end_header_id|> vb.) temizle
        temiz_cevap = cevap.replace("<|end_header_id|>", "").replace("<|eot_id|>", "").strip()

        return temiz_cevap
    except Exception as e:
        return f"Sistemde bir hata oluştu: {str(e)}"

# Arayüzün Görsel Tasarımı
with gr.Blocks(theme=gr.themes.Soft(primary_hue="blue", secondary_hue="slate")) as demo:
    gr.Markdown("""
    # 🚀 LLM Akademik Araştırma Asistanı
    ### Dr. Öğr. Üyesi Murat Şimşek - 2024-2025 Güz Dönemi Final Projesi
    **Veri Kaynağı:** 5 Akademik Makale (Transformer, RAG, GPT-3, Llama-3, ReAct) + 9 Ders Slaytı
    """)

    with gr.Row():
        with gr.Column(scale=2):
            soru_kutusu = gr.Textbox(
                label="Sorunuzu Buraya Yazın",
                placeholder="Örn: Transformer mimarisinde 'Encoder' ve 'Decoder' farkı nedir?",
                lines=4
            )
            gonder_btn = gr.Button("Analiz Et ve Yanıtla", variant="primary")
            gr.Examples(
                examples=["Self-attention mekanizması nedir?", "Kuantizasyonun avantajları nelerdir?", "RAG mimarisi nasıl çalışır?"],
                inputs=soru_kutusu
            )

        with gr.Column(scale=3):
            cevap_kutusu = gr.Textbox(
                label="Akademik Asistanın Türkçe Yanıtı",
                lines=12,
                interactive=False
            )

    gr.Markdown("---")
    gr.Markdown("ℹ️ *Bu proje Llama-3-8B modeli üzerinde 4-bit kuantizasyon ve LangChain RAG mimarisi kullanılarak geliştirilmiştir.*")

    # Buton tetikleyici
    gonder_btn.click(fn=akademik_asistan_yanitla, inputs=soru_kutusu, outputs=cevap_kutusu)

# Public URL oluşturur
demo.launch(share=True)

/tmp/ipython-input-3272346261.py:28: DeprecationWarning: The 'theme' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'theme' to Blocks.launch() instead.
  with gr.Blocks(theme=gr.themes.Soft(primary_hue="blue", secondary_hue="slate")) as demo:


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://8f25e7e6fb445bcc28.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
